In [236]:
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame 
import pandas_ta as pta
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn import svm

In [237]:
# Creating aplaca api key variables for access 
load_dotenv()
api_key = os.getenv('ALPACA_API_KEY')
api_secret = os.getenv('ALPACA_SECRET_KEY')

#creating timeframe to recieve most recent data
yesterday_date = date.today() - timedelta(1)


In [238]:
#Setting necessary variables for alpaca api function
base_url = 'https://paper-api.alpaca.markets'
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
account = api.get_account()

# Create the Alpaca API object
symbol= "ETHUSD"
ETH_df= {}

In [239]:
def get_bars(sym):
    test = api.get_crypto_bars(sym)
    return test


In [240]:
# Creating Function to clean data
def get_rsi(symbol:str = 'ETHUSD', yesterdays_date = date.today() - timedelta(1)):
    raw_data = api.get_crypto_bars(symbol,  TimeFrame.Day, "2017-06-08", yesterdays_date)
    raw_data = raw_data.df
    raw_data = raw_data.drop(columns=["open","high","low","volume","trade_count","vwap"])
    raw_data = raw_data[raw_data['exchange'].str.contains("ERSX|FTXU")==False]
    raw_data["Actual Returns"] = raw_data["close"].pct_change()
    raw_data["RSI EMA"] = pta.rsi(raw_data["close"], length = 30)
    raw_data["RSI Drift"] = pta.rsi(raw_data["close"], length = 30, drift = 10)
    raw_data = raw_data.dropna()
    raw_data["Signal"] = 0.0
    raw_data.loc[(ETH_df["Actual Returns"] >= 0), 'Signal'] = 1
    raw_data.loc[(ETH_df["Actual Returns"] < 0), 'Signal'] = -1
    raw_data['Strategy Returns'] = raw_data['Actual Returns'] * raw_data['Signal'].shift()
    
    return raw_data
    


In [242]:
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

In [241]:
def get_ml_indicator(df):
    X = df[["RSI EMA","RSI Drift" ]].shift().dropna()
    y = df['Signal']
    training_begin = X.index.min()
    training_end = X.index.min() + DateOffset(months=36)
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]
    X_test = X.loc[training_end+DateOffset(hours=1):]
    y_test = y.loc[training_end+DateOffset(hours=1):]
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    svm_model =svm.SVC()
     # Fit the model to the data using the training data
    svm_model = svm_model.fit(X_train_scaled, y_train) 
    # Use the testing data to make the model predictions
    svm_pred = svm_model.predict(X_test_scaled)
    test_out = df[training_end+DateOffset(hours=1):]
    test_out['ml'] = svm_pred
    # Review the model's predicted values 
    svm_pred[:10]
    return test_out



In [ ]:
while True
    ETH_df = get_rsi()
    rsi = ETH_df["RSI EMA"].loc[ETH_df.index.max()]
    ml_df = get_nl_indicator(ETH_df)
    ml_signal = ml_df["ml"].loc[ml_df.index.max()]
    if rsi >= 70 and ml_signal = -1:
        order_sell = api.submit_order(symbol='ETHUSD'
                                      ,qty=1
                                      ,side='sell'
                                      ,type='limit'
                                      ,time_in_force='gtc'
                                      ,limit_price=19000)
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif rsi <=30 and ml_signal = 1:
        order_buy = api.submit_order(symbol='ETHUSD'
                                     ,qty=1
                                     ,side='buy'
                                     ,type='market'
                                     ,time_in_force='gtc')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')
    time.sleep(get_pause())
    print("*"*20)
        
        
        
        
        
    